In [1]:
 # Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay, accuracy_score, precision_score, recall_score, f1_score
# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#pickle
import pickle

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#making the path to the csv and reading it in
filepath = "cleaned_data/clean_battle_info.csv"

battle_test = pd.read_csv(filepath)
print(battle_test.shape)
battle_test.head(10)

(2351, 8)


,Character,Strength,Speed,Intelligence,SpecialAbilities,Weaknesses,BattleOutcome,Universe
0,Wonder Woman,7,8,3,Telekinesis,Kryptonite,0,DC Comics
1,Iron Man,4,7,9,Telekinesis,Kryptonite,0,Marvel
2,Iron Man,8,7,5,Telekinesis,Magic,0,Marvel
3,Spider-Man,5,6,10,Telekinesis,Kryptonite,0,Marvel
4,Flash,7,6,2,Invisibility,Magic,0,DC Comics
5,Spider-Man,10,9,7,Invisibility,Wooden Stake,1,Marvel
6,Wonder Woman,3,6,2,Super Strength,Silver,1,DC Comics
7,Thor,7,2,4,Invisibility,Magic,1,Marvel
8,Batman,8,2,7,Flight,Silver,0,DC Comics
9,Iron Man,5,5,4,Flight,Wooden Stake,0,Marvel


In [3]:
battle_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2351 entries, 0 to 2350
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Character         2351 non-null   object
 1   Strength          2351 non-null   int64 
 2   Speed             2351 non-null   int64 
 3   Intelligence      2351 non-null   int64 
 4   SpecialAbilities  2351 non-null   object
 5   Weaknesses        2351 non-null   object
 6   BattleOutcome     2351 non-null   int64 
 7   Universe          2351 non-null   object
dtypes: int64(4), object(4)
memory usage: 147.1+ KB


In [4]:
# Encoding

battle_test["SpecialAbilities","Weaknesses","Character"] = battle_test.SpecialAbilities.astype(str)
battle_test = pd.get_dummies(battle_test, dtype=int)
battle_test.head()

,Strength,Speed,Intelligence,BattleOutcome,Character_Batman,Character_Captain America,Character_Flash,Character_Iron Man,Character_Spider-Man,Character_Superman,Character_Thor,Character_Wonder Woman,SpecialAbilities_Flight,SpecialAbilities_Invisibility,SpecialAbilities_Super Strength,SpecialAbilities_Telekinesis,Weaknesses_Kryptonite,Weaknesses_Magic,Weaknesses_Silver,Weaknesses_Wooden Stake,Universe_DC Comics,Universe_Marvel,"('SpecialAbilities', 'Weaknesses', 'Character')_Flight","('SpecialAbilities', 'Weaknesses', 'Character')_Invisibility","('SpecialAbilities', 'Weaknesses', 'Character')_Super Strength","('SpecialAbilities', 'Weaknesses', 'Character')_Telekinesis"
0,7,8,3,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1
1,4,7,9,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1
2,8,7,5,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1
3,5,6,10,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1
4,7,6,2,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0


In [5]:
# Scaler
scaler = pickle.load(open("battle_scaler.h5", 'rb'))

scaled_data = scaler.transform(battle_test.loc[:, ['Strength', 'Speed', 'Intelligence']])
battle_scaled2 = pd.DataFrame(scaled_data, columns=['Strength', 'Speed', 'Intelligence'])

# replace
battle_test["Strength"] = battle_scaled2.Strength
battle_test["Speed"] = battle_scaled2.Speed
battle_test["Intelligence"] = battle_scaled2.Intelligence

battle_test.head()

,Strength,Speed,Intelligence,BattleOutcome,Character_Batman,Character_Captain America,Character_Flash,Character_Iron Man,Character_Spider-Man,Character_Superman,Character_Thor,Character_Wonder Woman,SpecialAbilities_Flight,SpecialAbilities_Invisibility,SpecialAbilities_Super Strength,SpecialAbilities_Telekinesis,Weaknesses_Kryptonite,Weaknesses_Magic,Weaknesses_Silver,Weaknesses_Wooden Stake,Universe_DC Comics,Universe_Marvel,"('SpecialAbilities', 'Weaknesses', 'Character')_Flight","('SpecialAbilities', 'Weaknesses', 'Character')_Invisibility","('SpecialAbilities', 'Weaknesses', 'Character')_Super Strength","('SpecialAbilities', 'Weaknesses', 'Character')_Telekinesis"
0,0.537620,0.879933,-0.890985,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1
1,-0.498117,0.530073,1.204769,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1
2,0.882866,0.530073,-0.192401,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1
3,-0.152871,0.180213,1.554061,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1
4,0.537620,0.180213,-1.240278,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0


In [6]:
# model
model = pickle.load(open("battle_model.h5", 'rb'))

# columns in order
battle_test = battle_test.loc[:, ['Strength', 'Speed', 'Intelligence',
       'Character_Batman', 'Character_Captain America', 'Character_Flash',
       'Character_Iron Man', 'Character_Spider-Man', 'Character_Superman',
       'Character_Thor', 'Character_Wonder Woman', 'SpecialAbilities_Flight',
       'SpecialAbilities_Invisibility', 'SpecialAbilities_Super Strength',
       'SpecialAbilities_Telekinesis', 'Weaknesses_Kryptonite',
       'Weaknesses_Magic', 'Weaknesses_Silver', 'Weaknesses_Wooden Stake']]

preds = model.predict(battle_test)
print(preds[:200])

[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 1
 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 1 0 1
 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 1 1 0 0 0]
